In [5]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [9, 8]

import cv2
import numpy as np
import pandas as pd
pd.set_option('display.width', 74)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 20)

In [6]:
import os
import datetime
import itertools

from sklearn.metrics import confusion_matrix

DATADIR = "data/keras"
NUM_CHARACTERS = len(os.listdir(os.path.join(DATADIR, "train", "images")))
BATCH_SIZE = 16
LEARNING_RATE = 0.01

IMAGE_ROW_SIZE = 584
IMAGE_COLUMN_SIZE = 480

In [7]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input

def make_generator(folder="train",
                   data_gen_args={"fill_mode": "constant",
                                  "cval": 0,
                                  "width_shift_range": 0.05,
                                  "height_shift_range": 0.05,
                                  "zoom_range": 0.1,
                                  "horizontal_flip": True,
                                  "rescale": 1.0 / 255,
                                  "preprocessing_function": preprocess_input},
                   data_flow_args={"seed": 1,
                                   "batch_size": BATCH_SIZE}):

    image_datagen = ImageDataGenerator(**data_gen_args)

    image_generator = image_datagen.flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

    return image_generator


def count_images(folder="train"):
    image_directory = os.path.join(DATADIR, folder, "images")
    data_size = 0

    for char_name in os.listdir(image_directory):
        char_directory = os.path.join(image_directory, char_name)
        data_size += len(os.listdir(char_directory))

    return data_size


def steps_per_epoch(folder="train", batch_size=BATCH_SIZE):
    return count_images(folder) // batch_size

# Extract Features from my Images

We should not run this every time, instead just need to run it once to generate VGG16 features.

It takes a very long time to run.

In [8]:
from keras.applications.vgg16 import VGG16

model = VGG16(include_top=False, weights="imagenet")

To pull out all the images for feeding through VGG16 we use `class_mode=None` and `shuffled=False`.

In [9]:
image_count_train = count_images("train")
generator_train = make_generator(
    "train",
    data_gen_args={"preprocessing_function": preprocess_input},
    data_flow_args={"batch_size": BATCH_SIZE,
                    "class_mode": None,
                    "shuffle": False})
generator_train_labels = make_generator(
    "train",
    data_gen_args={},
    data_flow_args={"batch_size": BATCH_SIZE,
                    "shuffle": False})

Found 93638 images belonging to 8 classes.
Found 93638 images belonging to 8 classes.


This step takes super long.  I wish I could figure out a faster way to pull out just the labels in generator order.

In [20]:
labels_train = []
image_count_train_cutoff = image_count_train / BATCH_SIZE + 1
for i, (_, labels) in enumerate(generator_train_labels):
    if i >= image_count_train_cutoff:
        break
    else:
        labels_train.extend(labels)
labels_train = np.array(labels_train[:image_count_train])

Then this one takes a while too, but at least it makes sense why.  I wish I could figure out a faster way to do this as well.

In [ ]:
bottleneck_features_train = model.predict_generator(generator_train,
                                                    image_count_train_cutoff)
bottleneck_features_train = bottleneck_features_train[:image_count_train]

And now we save the results.

In [ ]:
with open(os.path.join(DATADIR, 'bottleneck_features_train.npy'), 'bw') as train:
    np.save(train, bottleneck_features_train)
with open(os.path.join(DATADIR, 'bottleneck_feature_labels_train.npy'), 'bw') as train:
    np.save(train, labels_train)

Do the same with our validation data.

In [ ]:
image_count_valid = count_images("valid")
generator_valid = make_generator(
    "valid",
    data_gen_args={"preprocessing_function": preprocess_input},
    data_flow_args={"batch_size": BATCH_SIZE,
                    "class_mode": None,
                    "shuffle": False})
generator_valid_labels = make_generator(
    "valid",
    data_gen_args={},
    data_flow_args={"batch_size": BATCH_SIZE,
                    "shuffle": False})

labels_valid = []
image_count_valid_cutoff = image_count_valid / BATCH_SIZE + 1
for i, (_, labels) in enumerate(generator_valid_labels):
    if i >= image_count_valid_cutoff:
        break
    else:
        labels_valid.extend(labels)
labels_valid = np.array(labels_valid[:image_count_valid])

bottleneck_features_valid = model.predict_generator(generator_valid,
                                                    image_count_valid_cutoff)
bottleneck_features_valid = bottleneck_features_valid[:image_count_valid]

with open(os.path.join(DATADIR, 'bottleneck_features_valid.npy'), 'bw') as valid:
    np.save(valid, bottleneck_features_valid)
with open(os.path.join(DATADIR, 'bottleneck_feature_labels_valid.npy'), 'bw') as valid:
    np.save(valid, labels_valid)

# Train a Small Neural Network

Load up our features and labels.

In [ ]:
with open(os.path.join(DATADIR, 'bottleneck_features_train.npy'), 'rb') as train:
    train_data = np.load(train)
with open(os.path.join(DATADIR, 'bottleneck_feature_labels_train.npy'), 'rb') as train:
    train_labels = np.load(train)

with open(os.path.join(DATADIR, 'bottleneck_features_valid.npy'), 'rb') as valid:
    valid_data = np.load(valid)
with open(os.path.join(DATADIR, 'bottleneck_feature_labels_valid.npy'), 'rb') as valid:
    valid_labels = np.load(valid)

We're gonna need these callbacks.

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

stop_on_val_loss = EarlyStopping(monitor='val_loss', min_delta=0.005, 
                                 patience=2, verbose=0, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, 
                              patience=2, min_lr=0.00001)

And, finally, start training our model.

In [ ]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dropout, Flatten, Dense

def init_model(input_shape, target_num=NUM_CHARACTERS,
               learning_rate=LEARNING_RATE):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(128, activation='relu', kernel_initializer='lecun_uniform'))
    model.add(Dense(target_num, activation='softmax'))

    model.compile(optimizer=Adam(lr=learning_rate),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    return model

In [ ]:
model = init_model(train_data.shape[1:], learning_rate=0.0001)

In [ ]:
model.summary()

In [ ]:
training_history = model.fit(train_data, train_labels,
                             epochs=50,
                             batch_size=BATCH_SIZE,
                             validation_data=(valid_data, valid_labels),
                             callbacks=[stop_on_val_loss, reduce_lr])

In [ ]:
model.save(os.path.join(DATADIR, datetime.datetime.today().strftime('%Y-%m-%d-%H-%M') + "-FIXED-DATA.h5"))

In [ ]:
def make_test_generator(folder="test",
                        data_flow_args={"seed": 1,
                                        "batch_size": BATCH_SIZE}):

    return ImageDataGenerator().flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

In [ ]:
test_batches = make_test_generator()
class_dict = test_batches.class_indices
index_dict = {i: c for c, i in class_dict.items()}

In [ ]:
def plot_confusion_matrix(matrix, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
images, vectors = next(test_batches)
true_indices = vectors.argmax(1)
predictions = model.predict_on_batch(images)
prediction_indices = predictions.argmax(1)
class_names = [c for i, c in sorted(index_dict.items())]

cnf_matrix = confusion_matrix(true_indices, prediction_indices)
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')